<a href="https://colab.research.google.com/github/YinGuoX/Deep_Learning_Pytorch_WithDeeplizard/blob/master/9_Creating_PyTorch_Tensors_For_Deep_Learning_Best_Options.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating PyTorch Tensors - Best Options

在这篇文章的最后，我们将知道主要选项之间的区别，以及哪些选项应该使用和何时使用。闲话少说，让我们开始吧。

我们已经看到的 PyTorch 张量是 torch.Tensor的实例。张量的抽象概念和 PyTorch 张量的区别在于，PyTorch 张量给我们提供了一个可以在代码中使用的具体实现。

在上一篇文章中，我们看到了如何在 PyTorch 中使用 Python list、 sequences 和 numpy.ndarray 等数据创建张量。给定一个 numpy.ndarray，我们发现有四种方法来创建一个torch.Tensor对象。

我们在这篇文章中的任务是探索这些选项之间的区别，并为我们的张量创建需求找到一个最佳选项。

**不同系统上的Numpy数据类型行为**

根据您的机器和操作系统，您的数据类型可能与此处显示的不同。

Numpy根据它是在32位系统上运行还是在64位系统上运行来设置它的默认数据类型，并且在Windows系统上的行为也有所不同。

此[链接](https://stackoverflow.com/questions/36278590/numpy-array-dtype-is-coming-as-int32-by-default-in-a-windows-10-64-bit-machine)提供了进一步的信息，可以对Windows系统上的差异进行重新分类。受影响的方法有：tensor、as_tensor和from_numpy。


## 1.张量创建操作: 有什么不同?
### 1.1 torch.Tensor() Vs toch.tesnor() 

第一个大写的torch.Tensor()是torch.Tensor的构造函数，第二个torch.tensor()是我们所说torch.Tensor对象的函数。构造torch.Tensor对象，并将它们返回给调用者。

您可以将 torch.tensor ()函数看作是一个工厂，在给定某些参数输入的情况下构建张量。工厂功能是用于创建对象的软件设计模式。如果你想了解更多，请点击[这里](https://en.wikipedia.org/wiki/Factory_(object-oriented_programming))。

好吧。这就是大写字母 t 和小写字母 t 的区别，但是哪种方式更好呢？答案是，两者都可以使用。

但是，工厂函数 torch.tensor ()有更好的文档和更多的配置选项，因此最好使用torch.tensor()。


### 1.2 默认dtype Vs Inferred dtype

当构造张量时，在torch.Tensor()构造函数使用默认的dtype。 我们可以使用torch.get_default_dtype()方法验证默认的dtype：

In [ ]:
import torch
torch.get_default_dtype()

torch.float32

为了验证代码，我们可以做到：

In [ ]:
import numpy as np
data = np.array([1,2,3])
o1 = torch.Tensor(data)
o2 = torch.tensor(data)
o3 = torch.as_tensor(data)
o4 = torch.from_numpy(data)
print(o1)

tensor([1., 2., 3.])


In [ ]:
o1.dtype==torch.get_default_dtype()

True

其他方法根据传入的数据选择出数据类型。这称为类型推断。数据类型是根据传入的数据推断出来的。

请注意，通过将dtype指定为参数，还可以为这些调用显式设置dtype：

In [ ]:
torch.tensor(data,dtype=torch.float64)

tensor([1., 2., 3.], dtype=torch.float64)

使用torch.Tensor()，我们无法将一个dtype传递给构造函数。 这是使用在torch.tensor()工厂函数来创建我们的张量的原因之一。

让我们来看看这些替代创建方法之间的最后一个隐藏区别。


### 1.3 Copy Vs Share

第三个区别是潜伏在幕后。为了揭示其中的差异，我们需要在使用 ndarray 创建张量之后对 numpy.ndarray 中的原始输入数据进行更改。

让我们这样做，看看我们得到了什么:


In [ ]:
print('old',data)

old [1 2 3]


In [ ]:
data[0]=100
print('new',data)

new [100   2   3]


In [ ]:
print(o1)
print(o2)
print(o3)
print(o4)


tensor([1., 2., 3.])
tensor([1, 2, 3])
tensor([100,   2,   3])
tensor([100,   2,   3])


**注意：**最初，我们有data [0] = 1，并且还注意到我们只更改了原始numpy.ndarray中的数据。我们并未明确更改张量（o1，o2，o3，o4）。

但是，在设置data [0] = 100后，我们可以看到一些张量发生了变化。前两个o1和o2仍具有原始值1，后两个o3和o4仍具有新值100。

发生这种情况是因为torch.Tensor（）和torch.tensor（）复制了它们的输入数据，而torch.as_tensor（）和torch.from_numpy（）与原始输入对象共享了它们在内存中的输入数据。

也即是：

| Share Data | Copy Data |
| :----: | :----: |
| torch.as_tensor() | torch.tensor() |
| torch.from_numpy() | torch.Tensor() |

这种共享仅仅意味着内存中的实际数据存在于一个地方。因此，基础数据中发生的任何更改都将反映在张量以及numpy.ndarra上.

与复制数据相比，共享数据效率更高，占用的内存更少，因为数据不会写入内存中的两个位置。

如果我们有torch.Tensor我们想把它转换成numpy.ndarray，我们可以这样做：


In [ ]:
print(o3.numpy())
print(type(o3.numpy()))

[100   2   3]
<class 'numpy.ndarray'>


这样可以确定torch.as_tensor（）和torch.from_numpy（）都与它们的输入数据共享内存。 但是，我们应该使用哪一个，它们有什么不同？

* torch.from_numpy（）函数仅接受numpy.ndarrays
* torch.as_tensor（）函数则接受各种类似数组的对象，包括其他PyTorch张量。 

因此，torch.as_tensor（）是内存共享中的优胜的选择。

## 2. 在PyTorch中创建张量的最佳选择

考虑到所有这些细节，这两个是最佳选择：

* torch.tensor()：Copy数据并且可以指定dtype
* torch.as_tensor()：Share数据并且接受更广的类型

**关于内存共享，需要记住的一些事情（它适用于任何地方）**：

* 因为numpy.ndarray对象是在CPU上分配的，当使用GPU时，as_tensor（）函数必须将数据从CPU复制到GPU。

* as_tensor（）的内存共享不适用于列表之类的内置Python数据结构。

* as_tensor（）调用要求开发人员了解共享功能。这是必要的，这样我们就不会无意中对基础数据进行不必要的更改，从而影响多个对象。

* 如果两个进程之间对numpy.ndarray和tensor对象有大量的来回操作，那么as_tensor（）的性能改进会更大。但是，如果只有一个加载操作，从性能的角度看应该不会有太大的影响。